## SpaRTaNS Tutorial: Double Chamber Flow
#### Last updated: 04/30/2022

## Export Inputs

#### Source Code

In [1]:
Needs["GeneralUtilities`"]
pythonSession = 
  StartExternalSession[<|"System" -> "Python", "Version" -> "3"|>];

SetDirectory[NotebookDirectory[]]

prepareInputs[sym_][geometry_] :=
 <|
  
  "A" -> <|
    
    "velocities" -> NumericArray[velocities[sym], "Real64"],
    "frequencies" -> NumericArray[frequencies[sym], "Real64"],
    "diagonal" -> NumericArray[diagonal[sym], "Real64"],
    "mixing_matrix" -> NumericArray[mixingMatrix[sym], "Real64"]
    |>,
  "000" -> <|
    
    "vertices" -> NumericArray[vertices[sym][geometry], "Real64"],
    "tetrahedra_indices" -> 
     NumericArray[tetrahedraIndices[sym][geometry], "Integer64"],
    "triangle_indices" -> 
     NumericArray[triangleIndices[sym][geometry], "Integer64"],
    If[normalsQ[sym][geometry], 
     "surface_normals" -> 
      NumericArray[normals[sym][geometry], "Real64"], Nothing]
    |>,
  "connectivity_info" -> <|
    
    "connectivity" -> 
     NumericArray[connectivity[sym][geometry], "Integer64"],
    "000--A_000--A" -> 
     Association[
      MapIndexed[
       StringTemplate["bounce_`1`"][
          IntegerString[First[#2] - 1, 10, 2]] -> 
         NumericArray[#1, "Integer64"] &, 
       triangleMappings[sym][geometry]]],
    "bounce_tensors" -> 
     Association[
      MapIndexed[
       StringTemplate["bounce_`1`"][
          IntegerString[First[#2] - 1, 10, 2]] -> 
         NumericArray[#1, "Real64"] &, bounceTensors[sym][geometry]]]
    |>,
  "000--A" -> <|
    
    If[surfaceInjectionQ[sym][geometry], 
     "surface_injection" -> 
      NumericArray[surfaceInjection[sym][geometry], "Real64"], 
     Nothing],
    If[bodyInjectionQ[sym][geometry], 
     "body_injection" -> 
      NumericArray[bodyInjection[sym][geometry], "Real64"], Nothing]
    |>
  |>

exportInputs[sym_][geometry_] := Block[{name, spartansInput},
  
  spartansInput = prepareInputs[sym][geometry];
  name = StringTemplate["spartans_test_`1`_`2`_dataset"][sym, 
    geometry];
  
  ExportStructuredHDF5[name <> ".h5", spartansInput];
  Run[StringTemplate[
     "h5repack -i `1`.h5 -o `1`-compressed.h5 -f GZIP=1"][name]];
  
  ExternalEvaluate[pythonSession, 
   StringTemplate[postProcessing[sym][geometry]][name, Directory[]]];
  
  Run[StringTemplate["rm `1`.h5"][name]]
  ]

- Finally, we export our inputs

In [ ]:
postProcessing["SO2"]["double-chamber"] = "
import h5py
import numpy as np
import os

with h5py.File('`2`/`1`-compressed.h5','a') as f:
    f['000--A'].attrs['material']='A'
    f['000--A'].attrs['mesh']='000'
    f['connectivity_info']['000--A_000--A'].attrs['outgoing_structure'\
]='000--A'
    f['connectivity_info']['000--A_000--A'].attrs['incoming_structure'\
]='000--A'
    f['connectivity_info']['connectivity'].attrs['ordering']=np.array(\
[b'000'])
";

In [ ]:
exportInputs["SO2"]["double-chamber"]